In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [2]:
from gemma_whisper import GemmaWhisperForConditionalGeneration
from transformers import AutoFeatureExtractor, AutoTokenizer, AutoProcessor
import librosa
import torch

torch.set_grad_enabled(False)

In [3]:
GemmaWhisperForConditionalGeneration.register_for_auto_class("AutoModel")

In [4]:
!ls gemma3n-audio-whisper-decoder-v2

checkpoint-112000  checkpoint-112500  checkpoint-113000  runs


In [5]:
model = GemmaWhisperForConditionalGeneration.from_pretrained(
    'gemma3n-audio-whisper-decoder-v2/checkpoint-113000', torch_dtype = 'auto')

In [6]:
_ = model.cuda()

In [7]:
feature_extractor = AutoFeatureExtractor.from_pretrained('mesolitica/gemma-3n-e4b-it-audio-encoder')
tokenizer = AutoTokenizer.from_pretrained('openai/whisper-large-v3-turbo')
processor = AutoProcessor.from_pretrained('openai/whisper-large-v3-turbo')

In [8]:
from streaming import MDSWriter, LocalDataset

dataset = LocalDataset('mosaic-stt')

In [11]:
i = 4000003
y, sr = librosa.load(dataset[i]['audio_filename'], sr = feature_extractor.sampling_rate)
input_ids = tokenizer(
    '<|startoftranscript|>', 
    add_special_tokens = False, return_tensors = 'pt')['input_ids']
features = feature_extractor([y], return_tensors = 'pt')
features['input_features'] = features['input_features'].cuda()
features['input_features_mask'] = features['input_features_mask'].cuda()
features['attention_mask'] = features['input_features_mask']
features['decoder_input_ids'] = input_ids.cuda()
generate_kwargs = dict(
    **features,
    max_new_tokens=1024,
    temperature=0.6,
    do_sample=True
)
generation_output = model.generate(**generate_kwargs)
tokenizer.decode(generation_output[0]), dataset[i]['text']

('<|startoftranscript|>Sopra le scarpe il nuove, prima o poi ci più ave.<|endoftext|>',
 'Sopra le scarpe nuove, prima o poi ci piove.')

In [ ]:
model.push_to_hub('mesolitica/gemma3n-audio-encoder-whisper-decoder')

In [ ]:
tokenizer.push_to_hub('mesolitica/gemma3n-audio-encoder-whisper-decoder')

In [ ]:
feature_extractor.push_to_hub('mesolitica/gemma3n-audio-encoder-whisper-decoder')